In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,to_date, date_format
from pyspark.sql import Row
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Assignment_2") \
    .config("spark.jars", "/Users/apple/Downloads/postgresql-42.7.0.jar")\
    .getOrCreate()

24/02/08 01:28:43 WARN Utils: Your hostname, Smritis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.87 instead (on interface en0)
24/02/08 01:28:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/08 01:28:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/08 01:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Reading the JSON file from local storage and converting it to a dataframe.
# createOrReplaceTempView() function creates a table name that can be used in spark-sql queries.

importDF= spark.read.json('/Users/apple/Downloads/review-Hawaii_10.json')
importDF.createOrReplaceTempView("RatingsAndReviews")
importDF.show(5)
importDF.count()

+--------------------+-----------------+----+------+----+-----------------+-------------+--------------------+
|             gmap_id|             name|pics|rating|resp|             text|         time|             user_id|
+--------------------+-----------------+----+------+----+-----------------+-------------+--------------------+
|0x7c00159b5b1b1d2...|  manuel grimaldo|NULL|     5|NULL|Great new upgrade|1591839903487|11396541707957662...|
|0x7c00159b5b1b1d2...|     Enrique Lara|NULL|     5|NULL|             NULL|1568059018979|11665581913729333...|
|0x7c00159b5b1b1d2...|Gregory Donaldson|NULL|     5|NULL|             NULL|1594885588335|10083411999455007...|
|0x7c00159b5b1b1d2...|      Brian Baker|NULL|     5|NULL|             NULL|1575951131613|10320721414448209...|
|0x7c00159b5b1b1d2...|            Kam J|NULL|     3|NULL|             NULL|1573076723916|10852617116317257...|
+--------------------+-----------------+----+------+----+-----------------+-------------+--------------------+
o

1504347

In [8]:
#A spark-sql query to convert the epoch time to date string with mm-dd-yyyy format

dateFormattedDF = spark.sql("SELECT *, date_format(to_date(from_unixtime(floor(time/1000))), 'MM-dd-yyyy') as date FROM RatingsAndReviews").drop("time").drop("pics").drop("resp")
dateFormattedDF.show(5)
dateFormattedDF.count()

+--------------------+-----------------+------+-----------------+--------------------+----------+
|             gmap_id|             name|rating|             text|             user_id|      date|
+--------------------+-----------------+------+-----------------+--------------------+----------+
|0x7c00159b5b1b1d2...|  manuel grimaldo|     5|Great new upgrade|11396541707957662...|06-11-2020|
|0x7c00159b5b1b1d2...|     Enrique Lara|     5|             NULL|11665581913729333...|09-10-2019|
|0x7c00159b5b1b1d2...|Gregory Donaldson|     5|             NULL|10083411999455007...|07-16-2020|
|0x7c00159b5b1b1d2...|      Brian Baker|     5|             NULL|10320721414448209...|12-10-2019|
|0x7c00159b5b1b1d2...|            Kam J|     3|             NULL|10852617116317257...|11-07-2019|
+--------------------+-----------------+------+-----------------+--------------------+----------+
only showing top 5 rows



1504347

In [9]:
# Create a table in postgre to load the dataframe. Use the following code to write the data to postgre server

url = "jdbc:postgresql://localhost:5432/postgres"
properties = { 
    "driver" : "org.postgresql.Driver",
    "user" : "vishal",
    "password" : "***********",
    "driver" : "org.postgresql.Driver",
    "batchsize" : "10000"
}

In [10]:
dateFormattedDF.write.jdbc(url=url, table = "Reviews", mode="overwrite", properties=properties)